In [ ]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import scipy.stats as stats
import pickle
import zipfile as zf

# import sys
# sys.path.append("../常模")
from func import get_angles_and_velocities, get_final_df

# 讀資料擴增後的檔案

In [ ]:
v_type = "abnormal"
aug_type = "v8"

folder_path = f"../20250615/data2Motion_{v_type}_clip_{aug_type}_all"
combined_data = {}

for filename in os.listdir(folder_path):
    if filename.endswith('.pkl'):
        file_path = os.path.join(folder_path, filename)
        # print(file_path)
        # 讀取 pickle 檔案
        with open(file_path, 'rb') as file:
            data = pickle.load(file)

        # filtered_data = {k: v for k, v in data.items() if k != "94_N_F_0.9Y_3(Mosaic)-json"}
        if "annotations" in data and len(data["annotations"]) > 0:
            frame_dir = data["annotations"][0].get("frame_dir")
            keypoint = data["annotations"][0].get("keypoint")

            if frame_dir is not None and keypoint is not None:
                combined_data[frame_dir] = keypoint

all_dir = combined_data.keys()
all_dir = list(all_dir)
print(all_dir)

In [ ]:
grouped_positions_dict = {}  # 儲存所有嬰兒的關鍵點數據

for idx, children_dir in tqdm(enumerate(all_dir), total=len(all_dir)):
    grouped_positions = []  # 單個嬰兒的關鍵點數據
    for frame in range(len(combined_data[all_dir[idx]][0])):
        temp = {}
        for j in range(len(combined_data[all_dir[idx]][0][frame])):
            grouped_positions_data = {j: list(combined_data[all_dir[idx]][0][frame][j].tolist())}
            temp.update(grouped_positions_data)
        grouped_positions.append(temp)

    grouped_positions_dict[children_dir] = grouped_positions

In [ ]:
# grouped_positions_dict['376_N_PWS_M_24.29WKS_5-0040']

# 開始計算指標

In [ ]:
all_angles_dict, all_velocities_dict = get_angles_and_velocities(grouped_positions_dict, all_dir, start_idx=0)

final_df = get_final_df(all_angles_dict, all_velocities_dict, all_dir, start_idx=0)

In [ ]:
for col in final_df.columns:
    if col.startswith("right_"):
        suffix = col[len("right_"):]  # 取得 like "elbow_angles"
        # print(suffix)
        left_col = f"left_{suffix}"
        print(col, left_col)
        if left_col in final_df.columns:
            lr_col = f"LR_{suffix}"
            final_df[lr_col] = (final_df[col] - final_df[left_col]).abs()